# ASTR 598 Astrostatistics
## HW2 Part 1
## Hayden Smotherman, Chris Suberlack, Winnie Wang
## To run this Notebook:

Run the file to calculate distances for part A and B. Part C is computed by both hand (manually) and with Astropy packages. However, due to the special handling of Astropy package, the computed number is slightly different with the  numerical computed value. Part D is obtained with respect to velocity dispersion.

In [ ]:
# Imports 
%matplotlib inline
from astropy.table import Table
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Galactocentric
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import CartesianDifferential
from astropy.coordinates import *
from astropy import units as u
from astropy.table import hstack
from astropy.table import vstack

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

import os
import numpy as np
import math
from astropy.io import fits
import pandas as pd
from scipy.stats import binned_statistic_2d as bs2d 
from scipy.stats import binned_statistic as bs1d

In [ ]:
# change given apparent and absolute magnitude into distance modulus: 
# m - M = 5*log(d) - 5
m = 21
M = 5
ra_deg = 0
dec_deg = 0

### Part A:

The heliocentric distance is simply obtained from changing the distance modulus into distance through the formula:
$(m - M) = 5*log(d) - 5$:

In [ ]:
d = 10 ** ((m - M + 5) / 5) #in parsecs
d_helio = (d/1000) * u.kpc

#print(d)
print("Heliocentric distance: "+ str(d_helio) + " for m = 21, Mr = 5")

### Part B:

To obtain the galactocentric distance to the halo stars that we are observing, we need to transform the heliocentric distance obtained from part A into galactocentric coordinate system, which we can use to change to distance.

In [ ]:
# Galactocentric coordinates:
# X = R − D cos(l) cos(b)
# Y = −D sin(l) cos(b)
# Z = D sin(b)
# where D = distance to the sun (heliocentric distance)
# distance = sqrt(X^2 + Y^2 + Z^2)

c = coord.ICRS(ra=[ra_deg] * u.degree, dec=[dec_deg] * u.degree, distance=[d_helio])
c2 = c.transform_to(coord.Galactocentric(galcen_distance=8.1*u.kpc)) #gives x,y,z

d_galacto = np.sqrt((c2.x)**2 + (c2.y)**2 + (c2.z)**2)

In [ ]:
d_galacto

### Part C:

To generate the radial velocity, the distributions were based on Bond et al. (2010) in section 5.1:

Setting up the velocity distributions:

In [ ]:
from astropy.coordinates import CylindricalRepresentation

def mean_velocity (seed, mean, sig, size):
    np.random.seed(seed) #resets seed
    mu = mean
    sigma = sig
    dist = np.random.normal(mu, sigma, size)
    dist_mean = np.mean(dist)
    
    for i in range(len(dist)):
        sum_squares = (dist[i] ** 2)
    
    dist_rms = np.sqrt((sum_squares) / len(dist))
    
    return dist_mean, dist_rms #returns mean and RMS of value

# mean and RMS radial velocity
radial_mean, radial_rms = (mean_velocity(42, 0, 135, 1000)) * (u.km / u.s) # mean is 0, std. dev is 135

# mean and RMS azimuthal velocity 
phi_mean, phi_rms = (mean_velocity(43, 0 ,85, 1000)) * (u.km / u.s) # mean is 0, std. dev is 85

# mean and RMS vertical velocity
vertical_mean, vertical_rms = (mean_velocity(44, 0, 85, 1000)) * (u.km / u.s) # mean is 0, std. dev is 85

Changing coordinates manually:

In [ ]:
coord = SkyCoord(ra=0*u.degree, dec=0*u.degree, frame='icrs') # coordinate of thing we are observing
galac_center = SkyCoord("17h45m37.2s -28d56m10.23s") # galactic center

# the following changes ra and dec to degrees
# reference: https://physics.stackexchange.com/questions/224950/how-can-i-convert-right-ascension-and-declination-to-distances
def change_to_theta(alpha, alpha_g, delta, delta_g):
    cos = (np.sin(delta))*(np.sin(delta_g)) + (np.cos(delta))*(np.cos(delta_g))*(np.sin(alpha - alpha_g))
    return np.arccos(cos)

theta = change_to_theta(coord.ra, galac_center.ra, 
                coord.dec, galac_center.dec) # angle between center of the galaxy and the thing I'm observing

# transform v_r, v_phi, v_z to v_x, v_y, v_z:
v_xx = (radial_mean) * np.cos(theta) + (phi_mean) * np.sin(theta)
v_yy = (radial_mean) * (-np.sin(theta)) + (phi_mean) * np.cos(theta)
v_zz = vertical_mean

# change (x,y,z) to (l,b)
coord2_2 = ICRS(x=-9.01548234*1000*u.pc, 
              y=7.83112784*1000*u.pc, 
              z=-13.72159591*1000*u.pc,
              v_x=v_xx, 
              v_y=v_yy, 
              v_z=v_zz,
              representation=CartesianRepresentation,
              differential_cls=CartesianDifferential) 

coord2_2.transform_to(LSR)

In [ ]:
coord2_2.transform_to(LSR).pm_ra_cosdec

In [ ]:
coord2_2.transform_to(LSR).pm_dec

Using Astropy packages (however the Astropy package is used incorrectly):

In [ ]:
# mean and RMS azimuthal velocity; units must be in radians for Astropy package to work

# mean and RMS azimuthal velocity 
phi_mean, phi_rms = (mean_velocity(43, 0 ,85, 1000)) * (u.km / u.s) # mean is 0, std. dev is 85

d_galacto_km = d_galacto.to(u.km)
one_km = 1 * u.km

angle = np.linalg.norm(one_km / d_galacto_km) * (u.rad / u.km) #radians / km

phi_mean_rad = (phi_mean * angle) * u.s 

cartesian_velocity = CylindricalRepresentation(radial_mean, 
                                               phi_mean_rad, vertical_mean).to_cartesian()
cartesian_vel = cartesian_velocity.norm()

coord2 = ICRS(x=-9.01548234*1000*u.pc, 
              y=7.83112784*1000*u.pc, 
              z=-13.72159591*1000*u.pc,
              v_x=cartesian_velocity.x, 
              v_y=cartesian_velocity.y, 
              v_z=cartesian_velocity.z,
              representation=CartesianRepresentation,
              differential_cls=CartesianDifferential)  

#coord2.transform_to(Galactic)

# change (l,b) to RA, Dec
coord2.transform_to(LSR)

In [ ]:
coord2.transform_to(LSR).pm_ra_cosdec

In [ ]:
coord2.transform_to(LSR).pm_dec

### Part D:

Median radial velocity:

In [ ]:
# median radial velocity is radial_mean
radial_mean

RMS radial velocity:

In [ ]:
# RMS radial velocity is radial_rms
radial_rms